#1.INSTALL NECESSARY **LIBRARIES**

In [1]:
! pip install selenium
! sudo apt-get update -y
! sudo apt install chromium-chromedriver -y
! pip install polars
! pip install xlsxwriter

! sudo cp /usr/lib/chromium-browser/chromedriver /usr/bin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 10.2 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,469 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,364 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy

#2. IMPORT NECESSARY LIBRARIES


In [2]:
import polars as pl
import pandas as pd
import selenium
import re
import sys
import time

# import selenium sub-modules and functions
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

#**3**.SETUP & FUNCTIONS



## i.Setup chrome driver

In [3]:
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.binary_location = '/usr/bin/chromium-browser'

chrome_options.add_argument("start-maximized")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)

wd = webdriver.Chrome(options=chrome_options )

#4.WEB-SCRAPE INFINIX PHONES LINKS

## i.Extract data from Homepage(sections, Link, Topics & Content)

In [4]:
wd = webdriver.Chrome(options=chrome_options )
url = "https://www.konga.com/search?search=infinix"  # Replace with the actual Konga URL for Infinix phones
wd.get(url)
print(wd.title)

  # Wait for the page to load
time.sleep(5)
links = wd.find_elements(By.XPATH,  "//a")
l = []
for i in links:
  l.append(i.get_attribute("href"))
  print(i.get_attribute("href"))
print(l)


Search results - infinix | Konga Online Shopping
https://www.konga.com/search?search=infinix#mainContent
https://www.konga.com/content/garnier
https://www.konga.com/search?search=infinix?mobile_menu=open
https://www.konga.com/
https://www.konga.com/stores
https://www.konga.com/cart/overview
https://www.konga.com/search?search=iphone
https://www.konga.com/search?search=phones
https://www.konga.com/search?search=laptop
https://www.konga.com/search?search=headphones
https://www.konga.com/product/konga-courier-bag-small-bundle-of-20-23cm-x-37cm-3576858
https://www.konga.com/product/konga-courier-bag-extra-large-bundle-of-20-51cm-x-63cm-3576884
https://www.konga.com/product/starlink-standard-actuated-kit-6574239
https://www.konga.com/product/hp-hp-652-black-original-ink-advantage-cartridge-f6v25ae-6424669
https://www.konga.com/product/itec-43-smart-led-full-hd-tv-usb-hdmi-dolby-1920-1080-screen-resolution-one-year-warranty-6571219
https://www.konga.com/brand/hp
https://www.konga.com/brand/s

##ii.Filtered link to infinix phone only

In [5]:
# To filtered links to infinix phones alone
filtered_link = []
for new_link in l:
# use Exception to ignore error
  try:
    if '/product/infinix' in new_link:
      filtered_link.append(new_link)
      print(new_link)
  except:
    pass

https://www.konga.com/product/infinix-infinix-s5-lite-tempered-glass-4795652?cid=5347
https://www.konga.com/product/infinix-infinix-s5-lite-tempered-glass-4795652?cid=5347
https://www.konga.com/product/infinix-infinix-xwatch-3-chic-green-6595912?cid=7576
https://www.konga.com/product/infinix-infinix-xwatch-3-chic-green-6595912?cid=7576
https://www.konga.com/product/infinix-infinix-note-6-temper-glass-4428150?cid=5347
https://www.konga.com/product/infinix-infinix-note-6-temper-glass-4428150?cid=5347
https://www.konga.com/product/infinix-infinix-note-6-back-case-4428149?cid=7581
https://www.konga.com/product/infinix-infinix-note-6-back-case-4428149?cid=7581
https://www.konga.com/product/infinix-smart-watch-1-1-69-ip68-waterproof-for-andriod-and-ios-bluetooth-callling-whi-6594797?cid=7576
https://www.konga.com/product/infinix-smart-watch-1-1-69-ip68-waterproof-for-andriod-and-ios-bluetooth-callling-whi-6594797?cid=7576
https://www.konga.com/product/infinix-smart-7-hd-x6516-64gb-ram-2gb-ro

#5.WEB-SCRAPE INFINIX PHONES

## i.Extract product_name from Konga websites

In [12]:
product_name = []
for link in filtered_link:
  wd = webdriver.Chrome(options=chrome_options )
  wd.get(link)
  time.sleep(3)
  title = wd.find_elements(By.XPATH, "/html/body/div[1]/div/section/div[3]/section/main/div[4]/div[3]/div[1]/div[1]/div/div[2]/div[2]/h4")
  for i in title:
    if len(i.text) == 0:
       product_name.append("None")
  else:
      print(i.text)
      product_name.append(i.text)

Infinix S5 Lite Tempered Glass
Infinix S5 Lite Tempered Glass
Infinix Xwatch 3 Chic Green
Infinix Xwatch 3 Chic Green
Infinix Note 6 Temper Glass
Infinix Note 6 Temper Glass
Infinix Note 6 Back Case
Infinix Note 6 Back Case
Infinix Smart Watch 1 - 1.69'' Ip68 Waterproof For Andriod & Ios - Bluetooth - Callling - Whi
Infinix Smart Watch 1 - 1.69'' Ip68 Waterproof For Andriod & Ios - Bluetooth - Callling - Whi
Infinix Smart 7 HD X6516 - 64GB RAM - 2GB ROM - 6000mAh - Green
Infinix Smart 7 HD X6516 - 64GB RAM - 2GB ROM - 6000mAh - Green
Infinix Xwatch 3 WE (XW3E) - Extra Stripe  - Magnetic Charger
Infinix Xwatch 3 WE (XW3E) - Extra Stripe  - Magnetic Charger
Infinix Xwatch 3plus - 2.13"amoled Curved Display With Extra Strap.
Infinix Xwatch 3plus - 2.13"amoled Curved Display With Extra Strap.
Infinix Gt20 Pro - 256GB ROM - 12GB RAM - Dual Sim -5g - Fingerprint - 5000mAh - Grey
Infinix Gt20 Pro - 256GB ROM - 12GB RAM - Dual Sim -5g - Fingerprint - 5000mAh - Grey
Infinix Smart 9 - X6532- 6.7

## ii.Extract product_brand from konga websites

In [14]:
product_brand = []
for link in filtered_link:
  wd = webdriver.Chrome(options=chrome_options )
  wd.get(link)
  time.sleep(3)
  brand = wd.find_elements(By.XPATH, "/html/body/div[1]/div/section/div[3]/section/main/div[4]/div[3]/div[1]/div[1]/div/div[2]/div[2]/form/div[2]/div[2]/a/span")
  for i in brand:
    if len(i.text) == 0:
       product_brand.append("None")
  else:
      print(i.text)
      product_brand.append(i.text)

Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix
Infinix


## iii.Extract product_price from konga websites

In [16]:
product_price = []
for link in filtered_link:
  wd = webdriver.Chrome(options=chrome_options )
  wd.get(link)
  time.sleep(3)
  price = wd.find_elements(By.XPATH, "/html/body/div[1]/div/section/div[3]/section/main/div[4]/div[3]/div[1]/div[1]/div/div[2]/div[2]/form/div[3]/div[1]/div[1]/div")
  for i in price:
    if len(i.text) == 0:
       product_price.append("None")
  else:
      print(i.text)
      product_price.append(i.text)

₦1,000
₦1,000
₦59,000
₦59,000
₦1,000
₦1,000
₦1,500
₦1,500
₦35,000
₦35,000
₦126,600
₦126,600
₦59,000
₦59,000
₦52,500
₦52,500
₦623,900
₦623,900
₦143,200
₦143,200
₦33,000
₦33,000
₦310,000
₦310,000
₦32,999
₦32,999
₦426,800
₦426,800
₦125,000
₦125,000
₦549,000
₦549,000
₦210,000
₦210,000
₦125,000
₦125,000
₦210,000
₦210,000
₦600,000
₦600,000
₦419,000
₦419,000
₦125,000
₦125,000
₦347,000
₦347,000
₦419,000
₦419,000
₦293,400
₦293,400
₦581,400
₦581,400
₦1,000
₦1,000
₦1,000
₦1,000
₦3,000
₦3,000


## iv.Extract product_code from konga websites

In [19]:
product_code = []
for link in filtered_link:
  wd = webdriver.Chrome(options=chrome_options )
  wd.get(link)
  time.sleep(3)
  code = wd.find_elements(By.XPATH, "/html/body/div[1]/div/section/div[3]/section/main/div[4]/div[3]/div[1]/div[1]/div/div[2]/div[2]/form/div[2]/div[1]/span")
  for i in code:
    if len(i.text) == 0:
       product_code.append("None")
  else:
      print(i.text)
      product_code.append(i.text)

4795652
4795652
6595912
6595912
4428150
4428150
4428149
4428149
6594797
6594797
6453914
6453914
6594795
6594795
6594796
6594796
6506878
6506878
6589403
6589403
6594829
6594829
6534724
6534724
6516793
6516793
6452038
6452038
6356488
6356488
6443243
6443243
6375223
6375223
6356485
6356485
6375221
6375221
6471257
6471257
6412396
6412396
6356489
6356489
6412395
6412395
6412398
6412398
6530330
6530330
6530323
6530323
4795651
4795651
4795650
4795650
4386123
4386123


# 6.VALIDATE THE LENGTH OF THE EXTRACTED DATA

In [39]:
print(len(product_name))
print(len(product_brand))
print(len(product_price))
print(len(product_code))
print(len(filtered_link))

58
58
58
58
58


# 7.CREATE A TABLE FOR THE EXTRACTED DATA

In [38]:
import pandas as pd

# Assuming 'df' is the DataFrame created in the previous code
# Create a dictionary to store the data
data = {
    'product_name': product_name,
    'product_brand': product_brand,
    'Product_price': product_price,
    'product_code': product_code,
    'product_link': filtered_link
}

# Create the Pandas DataFrame
df = pd.DataFrame(data)

# Convert the DataFrame to an HTML table
html_table = df.to_html()

#You can also display the table directly in a Jupyter Notebook or Colab environment:
from IPython.display import HTML
HTML(df.to_html())


,product_name,product_brand,Product_price,product_code,product_link
0,Infinix S5 Lite Tempered Glass,Infinix,"₦1,000",4795652,https://www.konga.com/product/infinix-infinix-s5-lite-tempered-glass-4795652?cid=5347
1,Infinix S5 Lite Tempered Glass,Infinix,"₦1,000",4795652,https://www.konga.com/product/infinix-infinix-s5-lite-tempered-glass-4795652?cid=5347
2,Infinix Xwatch 3 Chic Green,Infinix,"₦59,000",6595912,https://www.konga.com/product/infinix-infinix-xwatch-3-chic-green-6595912?cid=7576
3,Infinix Xwatch 3 Chic Green,Infinix,"₦59,000",6595912,https://www.konga.com/product/infinix-infinix-xwatch-3-chic-green-6595912?cid=7576
4,Infinix Note 6 Temper Glass,Infinix,"₦1,000",4428150,https://www.konga.com/product/infinix-infinix-note-6-temper-glass-4428150?cid=5347
5,Infinix Note 6 Temper Glass,Infinix,"₦1,000",4428150,https://www.konga.com/product/infinix-infinix-note-6-temper-glass-4428150?cid=5347
6,Infinix Note 6 Back Case,Infinix,"₦1,500",4428149,https://www.konga.com/product/infinix-infinix-note-6-back-case-4428149?cid=7581
7,Infinix Note 6 Back Case,Infinix,"₦1,500",4428149,https://www.konga.com/product/infinix-infinix-note-6-back-case-4428149?cid=7581
8,Infinix Smart Watch 1 - 1.69'' Ip68 Waterproof For Andriod & Ios - Bluetooth - Callling - Whi,Infinix,"₦35,000",6594797,https://www.konga.com/product/infinix-smart-watch-1-1-69-ip68-waterproof-for-andriod-and-ios-bluetooth-callling-whi-6594797?cid=7576
9,Infinix Smart Watch 1 - 1.69'' Ip68 Waterproof For Andriod & Ios - Bluetooth - Callling - Whi,Infinix,"₦35,000",6594797,https://www.konga.com/product/infinix-smart-watch-1-1-69-ip68-waterproof-for-andriod-and-ios-bluetooth-callling-whi-6594797?cid=7576


# 8.CLEAN DATA